In [10]:
#I will be using this notebook to build my final Capstone project
import pandas as pd
import numpy as np
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


In [45]:
#PART 1 -> Scraping the WIKI page and putting the findings into a dataframe.

from urllib.request import urlopen
import urllib.request
from bs4 import BeautifulSoup
import re

url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = urllib.request.urlopen(url)
soup = BeautifulSoup(page, "lxml")

#print(soup.prettify())
all_tables=soup.find_all("table")
all_tables

right_table = soup.find('table',class_='wikitable sortable')

PostalCode=[]
Borough=[]
Neighborhood=[]

for row in right_table.find_all('tr'):
    cells=row.find_all('td')
    if len(cells)==3:
        PostalCode.append(cells[0].text.strip())
        Borough.append(cells[1].text.strip())
        Neighborhood.append(cells[2].text.strip())

df=pd.DataFrame(PostalCode,columns=['PostalCode'])
df['Borough']=Borough
df['Neighborhood']=Neighborhood

#df['PostalCode'] = df['PostalCode'].astype('str')
#df['Borough'] = df['Borough'].astype('str')
#df['Neighborhood'] = df['Neighborhood'].astype('str')
df=df.astype('str')
df=df[df.Borough != 'Not assigned']
if df.Neighborhood is 'Not assigned':
    df.Neighborhood = Borough

group_df=df.groupby(['PostalCode','Borough'])['Neighborhood'].apply(','.join)
new_df=pd.DataFrame(group_df)
new_df2=new_df.reset_index()
new_df2['Neighborhood'] = new_df2['Neighborhood'].str.replace(' /',',')
new_df2.shape
new_df2

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park"
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge"
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [16]:
#PART 2 -> Find the long and lat of each neighborhood.

!pip install geocoder 
import geocoder # import geocoder

def get_geocoder(postal_code_from_df):
    lat_lng_coords = None
    
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Toronto, Ontario'.format(postal_code_from_df))
        lat_lng_coords=g.latlng
    neigh_lat = lat_lng_coords[0]
    neigh_lng = lat_lng_coords[1]
    return neigh_lat, neigh_lng

update_df=new_df2
update_df['Latitude'], update_df['Longitude'] = zip(*update_df['PostalCode'].apply(get_geocoder))

update_df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.808626,-79.189913
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.785779,-79.157368
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.765806,-79.185284
3,M1G,Scarborough,Woburn,43.771545,-79.218135
4,M1H,Scarborough,Cedarbrae,43.768791,-79.238813


In [17]:
#PART 3 -> Explore the findings (through the next few cells)

address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="tor_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Toronto are 43.6534817, -79.3839347.


In [20]:
#PART 3
!pip install folium
import folium # plotting library

In [21]:
#PART 3

# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(update_df['Latitude'], update_df['Longitude'], update_df['Borough'], update_df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [ ]:
#import pandas as pd 
#import requests 
#from bs4 import BeautifulSoup 

#req = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M") 
#soup = BeautifulSoup(req.content,'lxml') 
#table = soup.find_all('table')[0] 
#df = pd.read_html(str(table)) 

#neighborhood=pd.DataFrame(df[0])
#neighborhood

In [34]:
#PART 3

CLIENT_ID = '34TFTDFPIBKOQZ0BES1LEJHPMLTXY24C2WFK32C05BR54PIC' # your Foursquare ID
CLIENT_SECRET = 'HLLYL3IIB3J24UGFS1TKP2VIRDOYZSFG1TBFOJQMVYMD1EKR' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 100
radius = 500
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 34TFTDFPIBKOQZ0BES1LEJHPMLTXY24C2WFK32C05BR54PIC
CLIENT_SECRET:HLLYL3IIB3J24UGFS1TKP2VIRDOYZSFG1TBFOJQMVYMD1EKR


In [30]:
#PART 3
import requests
print("done")

done


In [35]:
#PART 3
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng, in zip (names, latitudes, longitudes):
        print(name)
    
    #API request
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
    
    #GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    #Return relevant info for each nearby venue
    venues_list.append([(
        name,
        lat,
        lng,
        v['venue']['name'],
        v['venue']['location']['lat'],
        v['venue']['location']['lng'],
        v['venue']['categories'][0]['name']) for v in results])
    
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [39]:
#PART 3
toronto_venues = getNearbyVenues(names=update_df['Neighborhood'],
                                   latitudes=update_df['Latitude'],
                                   longitudes=update_df['Longitude']
                                  )

Malvern, Rouge
Rouge Hill, Port Union, Highland Creek
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
Kennedy Park, Ionview, East Birchmount Park
Golden Mile, Clairlea, Oakridge
Cliffside, Cliffcrest, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Wexford Heights, Scarborough Town Centre
Wexford, Maryvale
Agincourt
Clarks Corners, Tam O'Shanter, Sullivan
Milliken, Agincourt North, Steeles East, L'Amoreaux East
Steeles West, L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
York Mills, Silver Hills
Willowdale, Newtonbrook
Willowdale
York Mills West
Willowdale
Parkwoods
Don Mills
Don Mills
Bathurst Manor, Wilson Heights, Downsview North
Northwood Park, York University
Downsview
Downsview
Downsview
Downsview
Victoria Village
Parkview Hill, Woodbine Gardens
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The Danforth West, Riverdale
India Bazaar, The Beaches West
Studio District
Lawrence P

In [37]:
#PART 3
print(toronto_venues.shape)
toronto_venues.head()

(2, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Northwest,43.711615,-79.588351,Woodbine Hotel & Suites,43.712624,-79.592664,Hotel
1,Northwest,43.711615,-79.588351,Economy Rent A Car,43.708471,-79.589943,Rental Car Location
